# 1. Getting the data

### The main code for sending an API request
After running this the object response will have the json we queried in pure text format.\
Some adjustments will be needed to get more than 20 items from the query, we can do this by using nextPageToken.

In [96]:
import requests, json
from secretfile import api_key

origin_location = "57.70867542111475,11.973303298399806"
radius_in_meters = 1000
query_string = "burger"

url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_string}&type=restaurant&location={origin_location}&radius={radius_in_meters}&key={api_key}"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
response.raise_for_status()


### Formatting the data we just got

We want to write our results to a file temporarily, then populate a dataframe with the data

In [59]:
import pandas as pd

In [60]:
with open('restaurants.json', 'w' , encoding='utf-8') as w:
    w.write(response.text)

In [92]:
# The keys function as attributes in this dict
result_dict = {"name": [], "rating": [], "user_ratings_total": [], "formatted_address": []}

# Populate the dict with values from the results
with open('restaurants.json', 'r', encoding='utf-8') as data_file:
    data = json.load(data_file)
    for restaurant in data['results']:
        result_dict['name'].append(restaurant['name'])
        result_dict['rating'].append(restaurant['rating'])
        result_dict['user_ratings_total'].append(restaurant['user_ratings_total'])
        result_dict['formatted_address'].append(restaurant['formatted_address'])

Now that we have the data we need to put it in a dataframe


In [95]:

# Create a neat dataframe and sort the rows by rating, user_ratings_total
df = pd.DataFrame.from_dict(result_dict)
df['rating'] = pd.to_numeric(df['rating'])
df['user_ratings_total'] = pd.to_numeric(df['user_ratings_total'])
df.sort_values(['rating', 'user_ratings_total'], ascending=False)


,name,rating,user_ratings_total,formatted_address
12,"DINÉ Burgers, femman huset",4.7,30,"Postgatan 26-31, 411 06 Göteborg"
3,The Barn,4.5,2393,"Kyrkogatan 11, 411 15 Göteborg"
1,2112,4.5,1913,"Magasinsgatan 5, 411 18 Göteborg"
17,Master Burger,4.5,53,"Södra Vägen 30, 412 54 Göteborg"
15,Johnny's Burger and Shakes,4.5,35,"Vasa Allen / avenue, Vasagatan 56, 411 36 Göte..."
4,DINÉ Burgers,4.4,926,"Drottninggatan 31, 411 14 Göteborg"
5,Tugg,4.3,2344,"Drottninggatan 21, 411 14 Göteborg"
0,Stockyard Burger Joint,4.3,888,"Storgatan 51, 411 38 Göteborg"
18,Barley's Food Factory,4.3,879,"Kvilletorget 20, 417 04 Göteborg"
19,Butcher's Market,4.3,827,"Storgatan 47, 411 38 Göteborg"


# 2. Analysing the data